In [1]:
import tensorflow as tf
import numpy as np
import scipy.sparse as sp
from scipy.sparse.linalg.eigen.arpack import eigsh
import scipy.sparse as sp
import pandas as pd
import pickle
from datetime import timedelta
import matplotlib.pyplot as plt
from scipy.stats.stats import pearsonr


In [2]:
file_Name = "../../data/nyc_bike/NYCBikeHourly272.pickle"
# open the file for writing
fileObject = open(file_Name,'rb') 
hourly_bike = pickle.load(fileObject) 
hourly_bike = pd.DataFrame(hourly_bike)
# results from bikedemandhourly_lstm_weather.ipynb
# new station id for the selected 272 stations
station_list = [0, 2, 3, 7, 8, 9, 11, 12, 27, 32, 33, 34, 38, 39, 40, 42, 43, 44, 46, 48, 52, 53, 54, 58, 59, 61, 62, 64, 65, 67, 68, 69, 74, 79, 80, 81, 82, 83, 84, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 104, 108, 111, 112, 115, 120, 121, 122, 124, 127, 130, 131, 133, 134, 135, 136, 138, 141, 142, 143, 145, 146, 150, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 169, 170, 171, 172, 173, 174, 176, 177, 178, 179, 180, 181, 182, 185, 186, 187, 188, 189, 195, 197, 198, 199, 200, 201, 204, 205, 206, 207, 208, 209, 210, 212, 214, 216, 219, 220, 221, 226, 242, 245, 249, 251, 252, 255, 257, 258, 263, 267, 273, 274, 275, 276, 280, 281, 282, 284, 290, 292, 295, 298, 299, 305, 306, 307, 308, 311, 316, 318, 320, 321, 322, 323, 327, 335, 340, 341, 342, 344, 345, 347, 348, 349, 353, 359, 361, 369, 371, 372, 393, 396, 397, 403, 404, 408, 410, 412, 414, 415, 416, 422, 429, 432, 434, 435, 438, 443, 446, 447, 448, 449, 450, 451, 452, 453, 455, 456, 457, 458, 459, 460, 467, 472, 475, 476, 478, 479, 480, 481, 482, 483, 484, 485, 486, 487, 488, 489, 490, 491, 492, 493, 495, 496, 497, 498, 499, 500, 501, 502, 503, 504, 506, 508, 509, 510, 511, 512, 514, 515, 516, 517, 518, 519, 520, 521, 522, 523, 534, 535, 537, 539, 540, 541, 542, 544, 545, 546, 547, 548, 549, 550, 553, 554]
new_old_stationID = pd.read_csv('../../data/nyc_bike/bike_stations.csv')
#new_old_stationID

In [3]:
new_old_stationID = new_old_stationID[new_old_stationID['station_id_new'].isin(station_list)]
#new_old_stationID

In [4]:
new_old_stationID = new_old_stationID.drop_duplicates('station_id_new', keep ='last')
new_old_stationID = new_old_stationID.reset_index()
#new_old_stationID

In [5]:
station_list = list(range(272))
#station_list

In [6]:
hourly_bike_gps = hourly_bike.transpose()
hourly_bike_gps['lat'] = new_old_stationID['lat']
hourly_bike_gps['lon'] = new_old_stationID['lon']
hourly_bike_gps['index1'] = range(272)

In [7]:
hourly_bike_gps.head()

,0,1,2,3,4,5,6,7,8,9,...,26297,26298,26299,26300,26301,26302,26303,lat,lon,index1
0,2,1,0,0,0,0,2,4,3,2,...,23,21,24,12,8,5,6,40.732219,-73.981656,0
1,1,0,0,0,0,0,1,1,4,13,...,3,12,18,11,8,4,1,40.741444,-73.975361,1
2,0,0,0,0,0,0,0,2,4,0,...,17,8,7,5,1,0,1,40.750020,-73.969053,2
3,3,1,0,1,1,0,0,2,7,4,...,0,0,0,0,0,0,0,40.750664,-74.001768,3
4,0,2,0,0,0,0,3,4,4,3,...,19,41,18,24,8,5,6,40.751396,-74.005226,4


In [8]:
def normalize_adj(adj):
    """Symmetrically normalize adjacency matrix."""
    #adj[np.isnan(adj)] = 0.
    adj = tf.abs(adj)
    rowsum = tf.reduce_sum(adj, 1)# sum by row

    d_inv_sqrt = tf.pow(rowsum, -0.5)
   
    #d_inv_sqrt[np.isinf(d_inv_sqrt)] = 0.
    
    d_mat_inv_sqrt = tf.diag(d_inv_sqrt)

    return tf.matmul(tf.matmul(d_mat_inv_sqrt, adj), d_mat_inv_sqrt)

In [9]:
# Create model
def gcn(x, weights, biases, batch_size, n_input, frequency,flag):
    # Hidden layer with RELU activation
    
    output_list = tf.Variable(tf.zeros([sn,1]),dtype=tf.float32) #'Tensor' object does not support item assignment, cant build Ypre
    
    if flag == 1:
        rownum = batch_size
    elif flag == 2:
        rownum = validation
    elif flag == 3:
        rownum = test
    
    for i in range(rownum):
        Xtem = tf.reshape(x[i,:], [frequency, n_input])
        Xtem = tf.transpose(Xtem)
        #Atem = tf.convert_to_tensor(A_whole_final, dtype=np.float32)
        #Atem1 = tf.diag(tf.ones([n_input])) # Atem not palying any roles
        #Ytem = tf.reshape(Y[i,:], [n_input, 1])
        Atem = tf.diag(tf.ones([n_input]))
        Atem1 = 0.5*(weights['A1'] + tf.transpose(weights['A1']))#+ Atem 
        Atem1 = normalize_adj(Atem1)
        #th = tf.constant(0.01, dtype=tf.float32)
        #where = tf.subtract(Atem1, th)
        #Atem1 = tf.nn.relu(where)
        
        Z1 = tf.matmul(Atem1, Xtem)
        #layer_1 = tf.matmul(Z1, weights['h1']) 
        layer_1 = tf.add(tf.matmul(Z1, weights['h1']), biases['b1'])
        layer_1 = tf.nn.relu(layer_1)
        
        #Atem2 = 0.5*(weights['A2'] + tf.transpose(weights['A2']))#+ Atem 
        #Atem2 = normalize_adj(Atem2)
        
        #Z2 = tf.matmul(Atem2, layer_1)
        #layer_2 = tf.add(tf.matmul(Z2, weights['h2']), biases['b2'])
        #layer_2 = tf.nn.relu(layer_2)
        
        #Atem3 = weights['A3']+ Atem 
        #Z3 = tf.matmul(Atem3, layer_2)
        #layer_3 = tf.add(tf.matmul(Z3, weights['h3']), biases['b3'])
        #layer_3 = tf.nn.relu(layer_3)
        
        # flattern
        #layer_3 = tf.reshape(layer_3, [1, 272*n_hidden_vec3])
        
        #F1 = tf.add(tf.matmul(layer_3, weights['f1']), biases['bf1'])
        #F1 = tf.nn.relu(F1)
        
        #F2 = tf.add(tf.matmul(F1, weights['f2']), biases['bf2'])
        #F2 = tf.nn.relu(F2)
        
        #F3 = tf.add(tf.matmul(F1, weights['f3']), biases['bf3'])
        #out_layer = tf.reshape(F3, [272, 1])

        # Output layer with linear activation
        Z4 = layer_1#tf.matmul(Atem1, layer_1)#tf.matmul(Atem, layer_3)
        out_layer = tf.add(tf.matmul(Z4, weights['out']), biases['bout'])
        
        # weather layer 1
        #x_wea_tem = tf.reshape(x_wea[i,:], [1, 9*frequency2]) # 1 by 126
        #layer_1_wea = tf.add(tf.matmul(x_wea_tem, weights['h1_wea']), biases['b1_wea'])
        #layer_1_wea = tf.nn.relu(layer_1_wea)
        
        #out_layer_wea = tf.add(tf.matmul(layer_1_wea, weights['out_wea']), biases['bout_wea'])
        #out_layer = tf.add(out_layer, tf.reshape(out_layer_wea, [272, 1]))
        
        #print (out_layer.get_shape())
        if i ==0:
            output_list = out_layer
        else:
            output_list = tf.concat([output_list, out_layer], 1)
        
        #print (tf.reduce_mean(tf.pow(output_list-out_layer, 2)))
    
    #print (output_list.get_shape())
    
    output_list = tf.transpose(output_list)
    
    #print (output_list.get_shape())
    
    return output_list

In [11]:
def gcn_corr_final(frequency, learning_rate, decay,batch_size, n_hidden_vec1,n_hidden_vec2,n_hidden_vec3,keep, early_stop_th,training_epochs, reg1, reg2):
    # set size
    #sn = 3 # station number

    frequency = int(frequency) #

    i = frequency
    X_whole = np.zeros(shape = (hourly_bike_cluster.shape[0] - frequency, sn*frequency), dtype = np.float)
    Y_whole = np.zeros(shape = (hourly_bike_cluster.shape[0] - frequency, sn), dtype = np.float)

    while i < hourly_bike_cluster.shape[0]:
        X_whole[i - frequency, ] = hourly_bike_cluster.iloc[(i - frequency):i, 0:sn].values.flatten() # flatten by row 0, 1, 2...7
        Y_whole[i - frequency, ] = hourly_bike_cluster.iloc[i, 0:sn]
        i = i + 1
        #print (i)
    
    skip = skip1 + freq_max - frequency # to make sure the testing datasets are the same although the frequency could be different

    X_training = X_whole[skip: skip+ training, :]
    Y_training = Y_whole[skip: skip+ training, :]
    #A_training = A_whole[0: 0+ training, :]

    X_val = X_whole[skip+training:skip+training+validation, :]
    Y_val = Y_whole[skip+training:skip+training+validation, :]
    #A_val = A_whole[0+training:0+training+validation, :]

    X_test = X_whole[skip+training+validation:skip+training+validation+test, :]
    Y_test = Y_whole[skip+training+validation:skip+training+validation+test, :]
    #A_test = A_whole[0+training+validation:0+training+validation+test, :]
    
    batch_size = int(batch_size)
    n_hidden_vec1 = int(n_hidden_vec1)
    n_hidden_vec2 = int(n_hidden_vec2)
    n_hidden_vec3 = int(n_hidden_vec3)
    early_stop_th = int(early_stop_th)
    training_epochs = int(training_epochs)
    
    early_stop_k=0
    display_step = 1
    best_val = 10000
    traing_error = 0
    test_error = 0
    # Network Parameters
    n_input = sn # station number
    n_input_vec = n_input * frequency # 317 * frequency
    n_A_vec = n_input * n_input
    n_output_vec = n_input * 1 # each row represent a result

    #n_classes = 2 # MNIST total classes (0-9 digits) # n_classes is for classification only

    tf.reset_default_graph()

    # tf Graph input
    X = tf.placeholder(tf.float32, [None, n_input_vec]) # X is the input signal
    #X_weather = tf.placeholder(tf.float32, [None, 9 * frequency2]) # X_weather weather and holiday information (9 is the feature number)
    A = tf.placeholder(tf.float32, [None, n_A_vec]) # A is the normalized adj matrix
    oldA = tf.placeholder(tf.float32, [n_input, n_input])
    Y = tf.placeholder(tf.float32, [None, n_output_vec]) # y is the regression output

    #Xtem = tf.placeholder(tf.float32, [n_input, frequency]) # for each row of X, A, Y, it can be reshaped to Xtem, Atem, Ytem
    #Atem = tf.placeholder(tf.float32, [n_input, n_input]) # 
    #Ytem = tf.placeholder(tf.float32, [n_input, 1]) #

    #Ypre = tf.placeholder(tf.float32, [None, n_output_vec])

    keep_prob = tf.placeholder(tf.float32) #dropout (keep probability)

    # Store layers weight & bias
    weights = {
        'h1': tf.Variable(tf.random_normal([frequency, n_hidden_vec1])),
        #'h2': tf.Variable(tf.random_normal([n_hidden_vec1, n_hidden_vec2])),
        #'h3': tf.Variable(tf.random_normal([n_hidden_vec2, n_hidden_vec3])),
        'out': tf.Variable(tf.random_normal([n_hidden_vec1, 1])), # dont forget to change n_hidden_vec1 when add/delete layers
        #'f1': tf.Variable(tf.random_normal([272*n_hidden_vec3, 100])),
        #'f2': tf.Variable(tf.random_normal([50, 10])),
        #'f3': tf.Variable(tf.random_normal([100, 272])),
        'A1': tf.Variable(tf.random_normal([n_input,n_input])),
        #'A2': tf.Variable(tf.random_normal([n_input,n_input])),
        #'A3': tf.Variable(tf.random_normal([n_input,n_input])),
        #'h1_wea': tf.Variable(tf.random_normal([9*frequency2, n_hidden_weather1])),
        #'out_wea': tf.Variable(tf.random_normal([n_hidden_weather1, n_input]))
    }
    biases = {
        'b1': tf.Variable(tf.random_normal([n_input,1])),# n_hidden_vec1])),# bias all the same??? dont forget to test
        #'b2': tf.Variable(tf.random_normal([n_input, 1])), #n_hidden_vec2])),
        #'b3': tf.Variable(tf.random_normal([n_input, 1])),#n_hidden_vec3])),
        #'b1': tf.Variable(tf.random_normal([n_input,n_hidden_vec1])),# bias all the same??? dont forget to test
        #'b2': tf.Variable(tf.random_normal([n_input,n_hidden_vec2])),
        #'b3': tf.Variable(tf.random_normal([n_input,n_hidden_vec3])),
        #'bf1': tf.Variable(tf.random_normal([1, 100])), 
        #'bf2': tf.Variable(tf.random_normal([1, 10])), 
        #'bf3': tf.Variable(tf.random_normal([1, 272])), 
        'bout': tf.Variable(tf.random_normal([n_input, 1])), 
        #'b1_wea': tf.Variable(tf.random_normal([1, n_hidden_weather1])), 
        #'bout_wea': tf.Variable(tf.random_normal([1, n_input])), 
    }

    #gcn_no_A
    #gcn
    # Construct model
    pred= gcn(X, weights, biases, batch_size,n_input, frequency, 1)
    
    #set  negative in pred to 0s
    #Computes rectified linear: max(features, 0).
    #pred = tf.nn.relu(pred)
    # Define loss and optimizer
    # RMS for regression 
    # L2 regularization
    #cost = tf.reduce_mean(tf.pow(pred-Y, 2)) + reg1*tf.nn.l2_loss(weights['A1']) + reg2*tf.nn.l2_loss(weights['A2'])# + tf.nn.l2_loss(weights['A2']) + tf.nn.l2_loss(weights['A3'])))

    # L1 regular tf.reduce_sum(tf.abs(parameters))
    cost = tf.reduce_mean(tf.pow(pred-Y, 2)) + reg1*tf.reduce_sum(tf.abs(weights['A1']))# + reg2*tf.reduce_sum(tf.abs(weights['A2']))
    pred_val= gcn(X, weights, biases, batch_size,n_input,frequency, 2)
    #pred_val = tf.nn.relu(pred_val)
    cost_val = tf.reduce_mean(tf.pow(pred_val-Y, 2))

    pred_tes= gcn(X, weights, biases, batch_size,n_input,frequency, 3)
    #pred_tes = tf.nn.relu(pred_tes)
    cost_tes = tf.reduce_mean(tf.pow(pred_tes-Y, 2))
    # cross-entropy for classification
    # cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=Y_train))
    # ratio = tf.abs(tf.reduce_sum(pred)-tf.reduce_sum(Y))/tf.reduce_sum(Y)
    #zero = 0
    #ratio = tf.reduce_mean(tf.divide(tf.where(tf.not_equal(Y, zero), np.abs(pred-Y), tf.zeros(Y.get_shape(), tf.float32)), tf.where(tf.not_equal(Y, zero), Y, tf.ones(Y.get_shape(), tf.float32))))
    optimizer = tf.train.RMSPropOptimizer(learning_rate, decay).minimize(cost)
    #optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

    # Initializing the variables
    init = tf.global_variables_initializer()
    saver = tf.train.Saver()

    #total_val_cost = []
    #total_val_ratio = []

    # learning start from 

    #index = daily_bike[(daily_bike['year'] == 2016) & (daily_bike['monthofyear'] == 1) & (daily_bike['dayofmonth'] == 1)].index.tolist()[0]
    #A_hat = normalize_adj(corr_matrix_trips)
    #print(A_hat)


    with tf.Session() as sess:
        sess.run(init)

        for epoch in range(training_epochs):

            avg_cost = 0.
            total_batch = int(training/batch_size)

            for i in range(total_batch):
                _, c = sess.run([optimizer, cost], feed_dict={X: X_training[i*batch_size:(i+1)*batch_size,], 
                                                      Y: Y_training[i*batch_size:(i+1)*batch_size,], 
                                                              keep_prob: keep})
                #print ("Epoch:", '%04d' % (epoch+1), "batch: ", i, "batch cost=", \
                #    "{:.9f}".format(c))
                #print (c)
                avg_cost += c / total_batch 
                #Display logs per epoch step
            if epoch % display_step == 0:
                print ("Epoch:", '%04d' % (epoch+1), "Training RMSE=", \
                    "{:.9f}".format((np.sqrt(avg_cost))))
            # validation
            c_val = sess.run([cost_val], feed_dict={X: X_val, Y: Y_val,  keep_prob:1})
            print("Validation RMSE: ", (np.sqrt(c_val[0])))

            c_tes, pred_tes1, A1= sess.run([cost_tes, pred_tes, weights['A1']], feed_dict={X: X_test,Y: Y_test, keep_prob: 1})
            print("Test RMSE: ", (np.sqrt(c_tes)))

            if c_val[0] < best_val:
                best_val = c_val[0]
                #saver.save(sess, './bikesharing_graph_2_th_point1')
                test_error = c_tes
                traing_error = np.sqrt(avg_cost)
                early_stop_k = 0 # reset to 0
                #print (pred_tes1)
                predic_res = pred_tes1

            # early stopping
            if c_val[0] >= best_val:
                early_stop_k += 1

            # threshold
            if early_stop_k == early_stop_th:
              #  print ("early stopping...")
                break
            

        print("epoch is ", epoch)
        print("training error is ", traing_error)
        print("Optimization Finished! the lowest validation RMSE is ", (np.sqrt(best_val)))
        print("The test RMSE is ", (np.sqrt(test_error)))
    
    test_Y = Y_test
    test_error = np.sqrt(test_error)
    return -np.sqrt(best_val), predic_res,test_Y,test_error, A1#, A2

In [12]:
# set size
#from bayes_opt import BayesianOptimization
import datetime

All_pred = np.empty([2000, 272])
All_Y = np.empty([2000, 272])

training = 20000
validation = 2000
test = 2000 #24*90
skip1 = 20000 + 2000 - training - validation # totally to make the testing dataset the same
freq_max = 24
#gcn_corr_eval(7, 0.01, 0.5, 100, 0.4, 10, 5, 5, 0.2, 50, 500)

step = 0
gap = 2000

total_sn = 0
num_iter = 50
init_points = 200
rep = 1000

# stdbscan
spatial_threshold = 300
temporal_threshold = 300
min_neighbors = 1 # number of neighbor

frequency2 = skip1 + freq_max + training

while step < 2000:
    
    #hourly_bike_gps_tem_index = hourly_bike_gps['index1']
    #hourly_bike_gps_tem = hourly_bike_gps.iloc[:, np.r_[skip1:frequency2, 26304:26306]] # past frequency2 steps and GPS locations
    # 0 index; 401:403 gps locations; ((i - frequency) + 1):(i + 1), temporal features
    #print (daily_bike_gps_tem)
    #df, cluster_num, station_in_cluster = ST_DBSCAN(hourly_bike_gps_tem, spatial_threshold, temporal_threshold, min_neighbors)

    #for c in range(cluster_num):
    #c1 = df[df['cluster'] == (c+1)].index.tolist()
    #sn = len(c1)
    #print ("this cluster size is: ", sn)
    #hourly_bike_cluster = hourly_bike[c1]
    #hourly_bike_cluster.shape

    sn = 272
    hourly_bike_cluster = hourly_bike
    '''
    gcnBO = BayesianOptimization(gcn_corr_eval, 
                                 {'frequency': (24, 24),
                                  'learning_rate': (0.01, 0.05),
                                    'decay': (0.3, 0.7),
                                    'batch_size': (80, 120),
                                    'n_hidden_vec1': (3, 10),
                                    'n_hidden_vec2': (3, 10),
                                    'n_hidden_vec3': (5, 5),
                                    'keep': (0.8, 1),
                                    'early_stop_th': (30, 70),
                                    'training_epochs': (500, 500),
                                    'reg': (0, 10)
                                    })
    gcnBO.maximize(init_points=init_points, n_iter=num_iter)
    a=gcnBO.res['max']['max_params']
    '''
    best = -10000
    pre_best = []
    test_Y_best = []
    test_error_best = 1000
    A1_best = []
   # A2_best = []
    for i in range(rep):
        a = datetime.datetime.now()
        val_error, predic_res, test_Y,test_error, A1=gcn_corr_final(24, 0.005, 0.9, 100, 40, 40, 5, 1, 20, 500, 0, 0)
        #val_error, predic_res, test_Y,test_error=gcn_corr_final(a['frequency'], a['learning_rate'], a['decay'], a['batch_size'], a['n_hidden_vec1'], a['n_hidden_vec2'], a['n_hidden_vec3'], a['keep'], a['early_stop_th'], a['training_epochs'], a['reg'])
        print ("finished A running: ", i)
        b = datetime.datetime.now()
        print(b-a)
        if val_error > best:
            best = val_error
            pre_best = predic_res
            test_Y_best = test_Y
            test_error_best = test_error
            A1_best = A1
           # A2_best = A2

    #val_error, predic_res, test_Y,test_error=gcn_corr_final(24, 0.02, 0.2, 100, 5, 10, 10, 0.8, 50, 500)
    All_pred[step:(step+gap), total_sn:(total_sn+sn)] = pre_best
    All_Y[step:(step+gap), total_sn:(total_sn+sn)]  = test_Y_best

    total_sn = total_sn + sn

    total_error = np.sqrt(np.mean((All_pred[0:(step+gap),0:total_sn] - All_Y[0:(step+gap),0:total_sn])**2))

    #print ("!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!the cluster now is: ", c)
    print ("!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!the val error of this cluster now is: ", best)
    print ("!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!the test error by this cluster now is: ", total_error)
    '''
    c1 = df[df['cluster'] == -999999].index.tolist()
    sn = len(c1)
    print ("this cluster size is: ", sn)
    hourly_bike_cluster = hourly_bike[c1]
    #hourly_bike_cluster.shape

    #sn = 272
    #hourly_bike_cluster = hourly_bike
    
    if sn != 0:
        
        gcnBO = BayesianOptimization(gcn_corr_eval, 
                                     {'frequency': (10, 24),
                                      'learning_rate': (0.01, 0.05),
                                        'decay': (0.1, 1),
                                        'batch_size': (50, 200),
                                        'n_hidden_vec1': (3, 20),
                                        'n_hidden_vec2': (5, 5),
                                        'n_hidden_vec3': (5, 5),
                                        'keep': (0.001, 1),
                                        'early_stop_th': (50, 500),
                                        'training_epochs': (200, 1000),
                                        'reg': (0, 100)
                                        })
        gcnBO.maximize(init_points=init_points, n_iter=num_iter)
        a=gcnBO.res['max']['max_params']
        
        best = -10000
        pre_best = []
        test_Y_best = []
        test_error_best = 1000
        for i in range(rep):
            #val_error, predic_res, test_Y,test_error=gcn_corr_final(24, 0.05, 0.5, 100, 5, 5, 5, 0.9, 50, 500, 0.5)
            val_error, predic_res, test_Y,test_error=gcn_corr_final(a['frequency'], a['learning_rate'], a['decay'], a['batch_size'], a['n_hidden_vec1'], a['n_hidden_vec2'], a['n_hidden_vec3'], a['keep'], a['early_stop_th'], a['training_epochs'], a['reg'])
            print ("finished A running: ", i)
            if val_error > best:
                best = val_error
                pre_best = predic_res
                test_Y_best = test_Y
                test_error_best = test_error

        #val_error, predic_res, test_Y,test_error=gcn_corr_final(24, 0.02, 0.2, 100, 5, 10, 10, 0.8, 50, 500)
        All_pred[step:(step+gap), total_sn:(total_sn+sn)] = pre_best
        All_Y[step:(step+gap), total_sn:(total_sn+sn)]  = test_Y_best

        total_sn = total_sn + sn

        total_error = np.sqrt(np.mean((All_pred[0:(step+gap),0:total_sn] - All_Y[0:(step+gap),0:total_sn])**2))

        print ("!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!the cluster now is: ", -999999)
        print ("!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!the val error of this cluster now is: ", best)
        print ("!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!the test error by this cluster now is: ", total_error)
    '''
    step = step + gap
    skip1 = skip1 + gap
    
    #np.savetxt("prediction_300_300.csv", All_pred, delimiter = ',')
    #np.savetxt("prediction_Y_300_300.csv", All_Y, delimiter = ',')
    

Epoch: 0001 Training RMSE= 14.504803800
Validation RMSE:  7.2411513
Test RMSE:  5.7073555
Epoch: 0002 Training RMSE= 7.038956559
Validation RMSE:  5.762159
Test RMSE:  4.468809
Epoch: 0003 Training RMSE= 6.173779911
Validation RMSE:  5.146015
Test RMSE:  3.9549868
Epoch: 0004 Training RMSE= 5.643278755
Validation RMSE:  4.782254
Test RMSE:  3.64464
Epoch: 0005 Training RMSE= 5.251906179
Validation RMSE:  4.4922132
Test RMSE:  3.4283123
Epoch: 0006 Training RMSE= 4.971628227
Validation RMSE:  4.315001
Test RMSE:  3.2988946
Epoch: 0007 Training RMSE= 4.781274176
Validation RMSE:  4.1704297
Test RMSE:  3.2050755
Epoch: 0008 Training RMSE= 4.623632702
Validation RMSE:  4.0570507
Test RMSE:  3.13657
Epoch: 0009 Training RMSE= 4.484992002
Validation RMSE:  3.9763865
Test RMSE:  3.08378
Epoch: 0010 Training RMSE= 4.369435874
Validation RMSE:  3.9036918
Test RMSE:  3.040478
Epoch: 0011 Training RMSE= 4.272511462
Validation RMSE:  3.82395
Test RMSE:  2.9990003
Epoch: 0012 Training RMSE= 4.19239

KeyboardInterrupt: 